In [1]:
# Add confidence interval
import config_cat_embedding
import pandas as pd
import numpy as np
import random
import time
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score)
from scipy import stats

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load and preprocess data
data_path = config_cat_embedding.paths['data']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')

df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
y = df_bank.y

# Convert target variable to numeric if necessary
# Assuming 'y' contains 'yes'/'no', map them to 1/0

# Define the classifiers
seed = 42
# We will determine the number_of_features inside the cross-validation loop after preprocessing

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', random_state=seed)),
    # ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(
        model=create_network,
        epochs=100, batch_size=100, verbose=0, random_state=seed))
]

# Cross-validation setup
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

# Function to calculate confidence intervals
def confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return m, m - h, m + h

# Loop over models
for name, model in models:
    print(f"Classifier: {name}")
    # Lists to store metrics for each fold
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    roc_aucs = []
    
    # Start the timer before cross-validation
    start_time = time.time()
    
    fold = 1
    for train_index, test_index in cv.split(X, y):
        # Split the data into training and testing sets for this fold
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        
        # Preprocess data within the fold
        # Hashing encoding
        import category_encoders as ce
        n_components = 9  
        hashing_encoder = ce.HashingEncoder(cols=cat_cols, n_components=n_components, return_df=True)
        X_train_hash = hashing_encoder.fit_transform(X_train_fold, y_train_fold)
        X_test_hash = hashing_encoder.transform(X_test_fold)
        
        # Standard scaling
        stc = StandardScaler()
        X_train_scaled = stc.fit_transform(X_train_hash)
        X_test_scaled = stc.transform(X_test_hash)
        
        # Update number_of_features for MLP
        number_of_features = X_train_scaled.shape[1]
        if name == 'MLP':
            # Update the model with the correct number of features
            model.set_params(model__number_of_features=number_of_features)
        
        # Fit the model
        model.fit(X_train_scaled, y_train_fold)
        # Predict on the test fold
        y_pred_fold = model.predict(X_test_scaled)
        
        # Get prediction probabilities for ROC AUC
        if hasattr(model, "predict_proba"):
            y_pred_prob_fold = model.predict_proba(X_test_scaled)[:, 1]
        else:
            print("no probability")
            # For classifiers without predict_proba, use decision_function
            y_pred_prob_fold = model.decision_function(X_test_scaled)
            # Normalize the decision function output to [0,1] range
            y_pred_prob_fold = (y_pred_prob_fold - y_pred_prob_fold.min()) / (y_pred_prob_fold.max() - y_pred_prob_fold.min())
        
        # Calculate metrics
        accuracies.append(accuracy_score(y_test_fold, y_pred_fold))
        precisions.append(precision_score(y_test_fold, y_pred_fold, zero_division=0))
        recalls.append(recall_score(y_test_fold, y_pred_fold))
        f1s.append(f1_score(y_test_fold, y_pred_fold))
        roc_aucs.append(roc_auc_score(y_test_fold, y_pred_prob_fold))
        
        # print(f"Fold {fold} completed.")
        fold += 1
    
    # Stop the timer after cross-validation
    end_time = time.time()
    total_computation_time = end_time - start_time  # Total time for the model
    
    # Calculate mean and confidence intervals
    acc_mean, acc_ci_lower, acc_ci_upper = confidence_interval(accuracies)
    prec_mean, prec_ci_lower, prec_ci_upper = confidence_interval(precisions)
    rec_mean, rec_ci_lower, rec_ci_upper = confidence_interval(recalls)
    f1_mean, f1_ci_lower, f1_ci_upper = confidence_interval(f1s)
    roc_mean, roc_ci_lower, roc_ci_upper = confidence_interval(roc_aucs)
    
    # Print results
    print(f"Accuracy: {acc_mean:.3f} (95% CI: {acc_ci_lower:.3f} - {acc_ci_upper:.3f})")
    print(f"Precision: {prec_mean:.3f} (95% CI: {prec_ci_lower:.3f} - {prec_ci_upper:.3f})")
    print(f"Recall: {rec_mean:.3f} (95% CI: {rec_ci_lower:.3f} - {rec_ci_upper:.3f})")
    print(f"F1 Score: {f1_mean:.3f} (95% CI: {f1_ci_lower:.3f} - {f1_ci_upper:.3f})")
    print(f"ROC AUC: {roc_mean:.3f} (95% CI: {roc_ci_lower:.3f} - {roc_ci_upper:.3f})")
    print(f"Total Computation Time: {total_computation_time:.3f} seconds\n")


Classifier: LR
Accuracy: 0.800 (95% CI: 0.797 - 0.803)
Precision: 0.667 (95% CI: 0.656 - 0.677)
Recall: 0.386 (95% CI: 0.375 - 0.397)
F1 Score: 0.489 (95% CI: 0.479 - 0.498)
ROC AUC: 0.845 (95% CI: 0.841 - 0.849)
Total Computation Time: 25.346 seconds

Classifier: DT
Accuracy: 0.798 (95% CI: 0.795 - 0.800)
Precision: 0.982 (95% CI: 0.977 - 0.986)
Recall: 0.187 (95% CI: 0.178 - 0.197)
F1 Score: 0.314 (95% CI: 0.301 - 0.327)
ROC AUC: 0.809 (95% CI: 0.805 - 0.813)
Total Computation Time: 27.934 seconds

Classifier: RF
Accuracy: 0.815 (95% CI: 0.812 - 0.818)
Precision: 0.954 (95% CI: 0.948 - 0.960)
Recall: 0.266 (95% CI: 0.256 - 0.276)
F1 Score: 0.416 (95% CI: 0.404 - 0.428)
ROC AUC: 0.872 (95% CI: 0.868 - 0.877)
Total Computation Time: 84.038 seconds

Classifier: KNN
Accuracy: 0.790 (95% CI: 0.787 - 0.793)
Precision: 0.587 (95% CI: 0.581 - 0.593)
Recall: 0.514 (95% CI: 0.503 - 0.525)
F1 Score: 0.548 (95% CI: 0.540 - 0.556)
ROC AUC: 0.782 (95% CI: 0.777 - 0.787)
Total Computation Time: 55.

In [1]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')

df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# One-hot encoding
import category_encoders as ce
hash_encoder=ce.HashingEncoder(cols=cat_cols,n_components=9)
X_train_ohe = hash_encoder.fit_transform(X_train)
X_test_ohe = hash_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)

number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

# Define the classifiers
seed = 42
models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', use_label_encoder=False)),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=lambda: create_network(number_of_features), epochs=100, batch_size=100, verbose=0))
]

# Train and evaluate the models
for name, model in models:
    start_time = time.time()  # Start time
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_pred_prob = model.predict_proba(X_test_scaled)
    
    end_time = time.time()  # End time
    computation_time = end_time - start_time  # Computation time
    
    print(f"Classifier: {name}")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))  # Set digits to 3 for three decimal places
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_prob[:, 1]):.3f}")
    print(f"Computation Time: {computation_time:.3f} seconds")
    print()

ModuleNotFoundError: No module named 'sklearn'

In [3]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone


# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')

df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# One-hot encoding
import category_encoders as ce
hash_encoder=ce.HashingEncoder(cols=cat_cols,n_components=9)
X_train_ohe = hash_encoder.fit_transform(X_train)
X_test_ohe = hash_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)
# Define the classifiers
seed = 42
number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss')),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=create_network, number_of_features=number_of_features, epochs=100, batch_size=100, verbose=0))
]

# Define number of runs for cross-validation
n_runs = 5
kf = StratifiedKFold(n_splits=n_runs, shuffle=True, random_state=seed)

# Initialize dictionary to store results
results = {}

# Loop over the models
for name, model in models:
    print(f"Classifier: {name}")
    
    # Lists to store metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    roc_auc_scores = []
    computation_times = []
    
    # Perform cross-validation
    for train_idx, test_idx in kf.split(X_train_scaled, y_train):
        # Clone the model to ensure fresh training for each fold
        clf = clone(model)
        
        X_train_fold, X_test_fold = X_train_scaled[train_idx], X_train_scaled[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        
        start_time = time.time()  # Start time
        
        clf.fit(X_train_fold, y_train_fold)
        y_pred = clf.predict(X_test_fold)
        y_pred_prob = clf.predict_proba(X_test_fold)[:, 1]  # Probabilities for ROC AUC
        
        end_time = time.time()  # End time
        computation_time = end_time - start_time  # Computation time
        
        # Append metrics for this fold
        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred))
        recall_scores.append(recall_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        roc_auc_scores.append(roc_auc_score(y_test_fold, y_pred_prob))
        computation_times.append(computation_time)
    
    # Compute mean and standard deviation of all metrics
    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)
    
    mean_precision = np.mean(precision_scores)
    std_precision = np.std(precision_scores)
    
    mean_recall = np.mean(recall_scores)
    std_recall = np.std(recall_scores)
    
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    mean_roc_auc = np.mean(roc_auc_scores)
    std_roc_auc = np.std(roc_auc_scores)
    
    mean_time = np.mean(computation_times)
    std_time = np.std(computation_times)
    
    # Store results for the current model
    results[name] = {
        "Mean Accuracy": mean_accuracy, "STD Accuracy": std_accuracy,
        "Mean Precision": mean_precision, "STD Precision": std_precision,
        "Mean Recall": mean_recall, "STD Recall": std_recall,
        "Mean F1 Score": mean_f1, "STD F1 Score": std_f1,
        "Mean ROC AUC": mean_roc_auc, "STD ROC AUC": std_roc_auc,
        "Mean Time": mean_time, "STD Time": std_time
    }
    
    # Print results
    print(f"Mean Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")
    print(f"Mean Precision: {mean_precision:.3f} ± {std_precision:.3f}")
    print(f"Mean Recall: {mean_recall:.3f} ± {std_recall:.3f}")
    print(f"Mean F1 Score: {mean_f1:.3f} ± {std_f1:.3f}")
    print(f"Mean ROC AUC: {mean_roc_auc:.3f} ± {std_roc_auc:.3f}")
    print(f"Mean Computation Time: {mean_time:.3f} ± {std_time:.3f} seconds")
    print()

Classifier: LR
Mean Accuracy: 0.800 ± 0.003
Mean Precision: 0.667 ± 0.006
Mean Recall: 0.392 ± 0.016
Mean F1 Score: 0.494 ± 0.013
Mean ROC AUC: 0.847 ± 0.004
Mean Computation Time: 0.031 ± 0.002 seconds

Classifier: DT
Mean Accuracy: 0.797 ± 0.001
Mean Precision: 0.980 ± 0.007
Mean Recall: 0.188 ± 0.005
Mean F1 Score: 0.315 ± 0.007
Mean ROC AUC: 0.811 ± 0.005
Mean Computation Time: 0.046 ± 0.005 seconds

Classifier: RF
Mean Accuracy: 0.815 ± 0.003
Mean Precision: 0.954 ± 0.014
Mean Recall: 0.267 ± 0.010
Mean F1 Score: 0.417 ± 0.014
Mean ROC AUC: 0.874 ± 0.005
Mean Computation Time: 2.495 ± 0.044 seconds

Classifier: KNN
Mean Accuracy: 0.790 ± 0.003
Mean Precision: 0.587 ± 0.009
Mean Recall: 0.515 ± 0.004
Mean F1 Score: 0.549 ± 0.005
Mean ROC AUC: 0.783 ± 0.007
Mean Computation Time: 7.481 ± 0.726 seconds

Classifier: XGB
Mean Accuracy: 0.840 ± 0.006
Mean Precision: 0.740 ± 0.021
Mean Recall: 0.549 ± 0.011
Mean F1 Score: 0.630 ± 0.012
Mean ROC AUC: 0.898 ± 0.004
Mean Computation Time: 0

C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87

Mean Accuracy: 0.818 ± 0.003
Mean Precision: 0.669 ± 0.016
Mean Recall: 0.532 ± 0.041
Mean F1 Score: 0.591 ± 0.020
Mean ROC AUC: 0.872 ± 0.004
Mean Computation Time: 47.684 ± 3.187 seconds

